## NeuroDet: LSTM Model

### Import Libraries

In [9]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import PIL
import PIL.Image
import matplotlib.pyplot as plt
import pathlib
import math

# Disabling GPU for the moment because of the lack of the memory
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


### Loading the dataset of Brain Scan images
#### Full data for training and testing
Source of the Dataset: [Kaggle-Brain Tumor Classification](https://www.kaggle.com/sartajbhuvaji/brain-tumor-classification-mri?select=Testing)<br>
Reference for operations performed : [Tensorflow tutorial: Load Images](https://www.tensorflow.org/tutorials/load_data/images)

In [10]:
categories_path = {'glioma_tumor': '/glioma_tumor', 'meningioma_tumor': '/meningioma_tumor', 
                   'pituitary_tumor': '/pituitary_tumor', 'no_tumor':'/no_tumor'}
train_path = 'BrainMRI/Training'
test_path = 'BrainMRI/Testing'

# train_glioma_dir = pathlib.Path(train_path + categories_path['glioma_tumor'])
train_dir = pathlib.Path(train_path)
test_dir = pathlib.Path(test_path)

# Training data: number of examples
# label = 0 | glioma_tumor: 826
# label = 1 | meningioma_tumor: 822
# label = 2 | no_tumor: 395 
# label = 3 | pituitary_tumor: 827 

# Testing data: number of examples
# label = 0 | glioma_tumor: 100
# label = 1 | meningioma_tumor: 115
# label = 2 | no_tumor: 105
# label = 3 | pituitary_tumor: 74

num_train_examples_dict = {
    "label_0": 826, "label_1": 822, "label_2": 395, "label_3": 827}

num_test_examples_dict = {
    "label_0": 100, "label_1": 115, "label_2": 105, "label_3": 74}


num_train_examples = sum(num_train_examples_dict.values()) # 2870
num_test_examples = sum(num_test_examples_dict.values()) # 394

print(f'Total number of train examples: {num_train_examples}')
print(f'Total number of test examples: {num_test_examples}') 

# Defining the parameters of the dataset
img_height = 256
img_width = 256 

Total number of train examples: 2870
Total number of test examples: 394
Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
Found 2870 files belonging to 2 classes.
Found 394 files belonging to 2 classes.
['no_tumor', 'tumor']
['no_tumor', 'tumor']


#### 4-Class Classification Data 

In [ ]:
# Loading the train dataset using keras.utils.image_dataset_from_directory
# To use this method, please ensure you have tf.nigthly installed 

train_data_full = tf.keras.utils.image_dataset_from_directory(
    train_dir, seed=123,
    image_size=(img_height, img_width),
    batch_size=num_train_examples, 
    shuffle=True,
    color_mode = 'grayscale')

test_data_full = tf.keras.utils.image_dataset_from_directory(
    test_dir, seed=123,
    image_size=(img_height, img_width),
    batch_size=num_test_examples, 
    shuffle=True,
    color_mode = 'grayscale')

print(train_data_full.class_names)
print(test_data_full.class_names)

#### 2-Class Classification Data

In [ ]:
train2_path = 'BrainMRI_2Class/Training'
test2_path = 'BrainMRI_2Class/Testing'

train2_dir = pathlib.Path(train2_path)
test2_dir = pathlib.Path(test2_path)

train2_data_full = tf.keras.utils.image_dataset_from_directory(
    train2_dir, seed=123,
    image_size=(img_height, img_width),
    batch_size=num_train_examples, 
    color_mode = 'grayscale')

test2_data_full = tf.keras.utils.image_dataset_from_directory(
    test2_dir, seed=123,
    image_size=(img_height, img_width),
    batch_size=num_test_examples, 
    color_mode = 'grayscale')

print(train2_data_full.class_names)
print(test2_data_full.class_names)

### Normalizing the training and testing set

In [11]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_data_full = train_data_full.map(lambda x, y: (normalization_layer(x), y))
test_data_full = test_data_full.map(lambda x, y: (normalization_layer(x), y))

train2_data_full = train2_data_full.map(lambda x, y: (normalization_layer(x), y))
test2_data_full = test2_data_full.map(lambda x, y: (normalization_layer(x), y))

### Separate into images and labels for each set

In [12]:
for images, labels in train_data_full:
    x_train = images
    y_train = labels
    
for images, labels in test_data_full:
    x_test = images
    y_test = labels
    
for images, labels in train2_data_full:
    x_train2 = images
    y_train2 = labels
    
for images, labels in test2_data_full:
    x_test2 = images
    y_test2 = labels

# 4 Class LSTM Model

In [ ]:
# Parameters
lstm1_size = 128
num_epochs = 30
learning_rate = 0.006
dense1_size = 80
dense2_size = 4
dropout_rate = 0.2


# LSTM model
lstm_model = tf.keras.Sequential()
lstm_model.add(tf.keras.layers.Input(batch_input_shape=(None,img_height,img_width)))
lstm_model.add(tf.keras.layers.LSTM(lstm1_size))
lstm_model.add(tf.keras.layers.Dense(dense1_size, activation='relu'))
lstm_model.add(tf.keras.layers.Dense(dense2_size, activation='softmax'))


lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

lstm_model.fit(
    tf.squeeze(x_train), 
    tf.squeeze(y_train), 
    epochs=num_epochs)

lstm_model.summary()

test_loss, test_accuracy = lstm_model.evaluate(x_test, y_test)

print(f'Testing loss = {test_loss}, Testing accuracy = {test_accuracy}')

# 2 Class LSTM Model

In [15]:
# Parameters
lstm1_size = 128
num_epochs = 20
learning_rate = 0.005
dense1_size = 50
dense2_size = 2


# LSTM model
lstm_model = tf.keras.Sequential()
lstm_model.add(tf.keras.layers.Input(batch_input_shape=(None,img_height,img_width)))
lstm_model.add(tf.keras.layers.LSTM(lstm1_size))
lstm_model.add(tf.keras.layers.Dense(dense1_size, activation='relu'))
lstm_model.add(tf.keras.layers.Dense(dense2_size, activation='softmax'))


lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

lstm_model.fit(
    tf.squeeze(x_train2), 
    tf.squeeze(y_train2), 
    epochs=num_epochs)

lstm_model.summary()

test_loss, test_accuracy = lstm_model.evaluate(x_test2, y_test2)

print(f'Testing loss = {test_loss}, Testing accuracy = {test_accuracy}')

Epoch 1/20
90/90 [==============================] - 31s 302ms/step - loss: 0.3411 - accuracy: 0.8679
Epoch 2/20
90/90 [==============================] - 27s 301ms/step - loss: 0.2422 - accuracy: 0.8920
Epoch 3/20
90/90 [==============================] - 28s 311ms/step - loss: 0.2004 - accuracy: 0.9094
Epoch 4/20
90/90 [==============================] - 28s 311ms/step - loss: 0.1712 - accuracy: 0.9261
Epoch 5/20
90/90 [==============================] - 28s 311ms/step - loss: 0.1407 - accuracy: 0.9432
Epoch 6/20
90/90 [==============================] - 28s 307ms/step - loss: 0.1329 - accuracy: 0.9425
Epoch 7/20
90/90 [==============================] - 28s 312ms/step - loss: 0.1170 - accuracy: 0.9495
Epoch 8/20
90/90 [==============================] - 27s 299ms/step - loss: 0.1198 - accuracy: 0.9474
Epoch 9/20
90/90 [==============================] - 28s 306ms/step - loss: 0.1056 - accuracy: 0.9564
Epoch 10/20
90/90 [==============================] - 27s 304ms/step - loss: 0.1027 - accura